<a href="https://colab.research.google.com/github/samarakoon-ryan/creditcard-fraud-detection/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit Card Fraud Detection

In [1]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

## Exploratory Data Analysis

In [2]:
# Read dataset

detection = pd.read_csv("creditcard.csv")
detection.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [6]:
# Summary statistics for each feature
detection.describe().T

,count,mean,std,min,25%,50%,75%,max
Time,188936.0,67129.863361,32342.234667,0.000000,43140.500000,64902.000000,84467.000000,128198.000000
V1,188936.0,-0.125252,1.881109,-56.407510,-0.969170,-0.118625,1.202954,2.439207
V2,188936.0,0.013083,1.618539,-72.715728,-0.569894,0.087736,0.787778,22.057729
V3,188936.0,0.378330,1.429868,-33.680984,-0.240420,0.538723,1.250673,9.382558
V4,188936.0,0.087745,1.390879,-5.683171,-0.778484,0.090206,0.904313,16.875344
V5,188936.0,-0.134218,1.356787,-42.147898,-0.799850,-0.189658,0.433807,34.801666
V6,188936.0,0.051501,1.306893,-26.160506,-0.703126,-0.212980,0.446274,22.529298
V7,188936.0,-0.061331,1.213400,-43.557242,-0.580461,-0.018419,0.486489,36.677268
V8,188936.0,0.022442,1.214328,-73.216718,-0.174861,0.047507,0.342231,20.007208
V9,188936.0,0.012329,1.140722,-13.434066,-0.661821,-0.075132,0.630751,15.594995
